# Simplest 1D Gan
Simplest example based on this tutorial:
* [Blog post](http://blog.aylien.com/introduction-generative-adversarial-networks-code-tensorflow/)
* [Code](https://github.com/AYLIEN/gan-intro)

In [1]:
import numpy as np
import tensorflow as tf

# Include modules from other directories
import sys
sys.path.append('../tensorflow/')
import model_util as util
import anim_util as anim

import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

# Fix seed to reproduce same results
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

# Some meta parameters
HIDDEN_SIZE=4
start_lr = 0.005
decay = 0.95
num_steps = 5000
batch_size = 8
num_decay_steps = 150
logs_path = './logs'
save_dir = './save'
gpu_fraction = 0.1

# Delete logs directory if exist
if os.path.exists(logs_path):    
    print('Older log exist deleting ...')
    os.system("rm -rf " + logs_path)

In [2]:
class DataDistribution(object):
    def __init__(self):
        self.mu = 4
        self.sigma = 0.5

    def sample(self, N):
        samples = np.random.normal(self.mu, self.sigma, N)
        samples.sort()
        return samples


class GeneratorDistribution(object):
    def __init__(self, range):
        self.range = range

    def sample(self, N):
        return np.linspace(-self.range, self.range, N) + np.random.random(N) * 0.01    

### Define Generator and Discriminator models
Observe that the discriminator on this example is more powerfull than the generator.
* https://en.wikipedia.org/wiki/Rectifier_(neural_networks)#/media/File:Rectifier_and_softplus_functions.svg

In [3]:
def generator(input, hidden_size):
    h0 = tf.nn.softplus(util.linear_std(input, hidden_size, 'g0'))
    h1 = util.linear_std(h0, 1, 'g1')
    return h1

def discriminator(input, hidden_size):
    h0 = tf.tanh(util.linear_std(input, hidden_size * 2, 'd0'))
    h1 = tf.tanh(util.linear_std(h0, hidden_size * 2, 'd1'))
    h2 = tf.tanh(util.linear_std(h1, hidden_size * 2, 'd2'))
    h3 = tf.sigmoid(util.linear_std(h2, 1, 'd3'))
    return h3

### Define GAN

In [4]:
with tf.variable_scope('G'):
    # z is the generator vector input
    z = tf.placeholder(tf.float32, shape=(None, 1))
    # G will have the generator output tensor
    G = generator(z, HIDDEN_SIZE)

with tf.variable_scope('D') as scope:
    x = tf.placeholder(tf.float32, shape=(None, 1))
    D_real = discriminator(x, HIDDEN_SIZE)
    scope.reuse_variables()
    D_fake = discriminator(G, HIDDEN_SIZE)

with tf.variable_scope('loss'):
    # Define losses
    loss_d = tf.reduce_mean(-tf.log(D_real) - tf.log(1 - D_fake))
    loss_g = tf.reduce_mean(-tf.log(D_fake))

### Get parameters from Generator and Discriminator

In [5]:
vars = tf.trainable_variables()
d_params = [v for v in vars if v.name.startswith('D/')]
g_params = [v for v in vars if v.name.startswith('G/')]

### Create the Session
Basically ask tensorflow to build the graph

In [6]:
# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

### Define the solver
We want to use the Adam solver to minimize or loss function.

In [7]:
def optimizer(loss, var_list, name='Solver'):
    # Solver configuration
    # Get ops to update moving_mean and moving_variance from batch_norm
    # Reference: https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.name_scope(name):
        global_step = tf.Variable(0, trainable=False)
        starter_learning_rate = start_lr
        # decay every 10000 steps with a base of 0.96
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                   num_decay_steps, decay, staircase=True)

        # Basically update the batch_norm moving averages before the training step
        # http://ruishu.io/2016/12/27/batchnorm/
        with tf.control_dependencies(update_ops):
            train_step = tf.train.AdamOptimizer(
                learning_rate).minimize(loss, global_step=global_step, var_list=var_list)
    
    return train_step


In [8]:
opt_disc = optimizer(loss_d, d_params, 'Solver_Disc')
opt_gen = optimizer(loss_g, g_params, 'Solver_Gen')

### Add some variables to tensorboard

In [9]:
# Create histogram for labels
tf.summary.histogram("data_dist", x)
tf.summary.histogram("latent", z)

# Monitor loss, learning_rate, global_step, etc...
tf.summary.scalar("loss_disc", loss_d)
tf.summary.scalar("loss_gen", loss_g)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Configure where to save the logs for tensorboard
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

### Initialize the values (Random values of weights)

In [10]:
# Initialize all random variables (Weights/Bias)
sess.run(tf.global_variables_initializer())

In [11]:
data = DataDistribution()
gen = GeneratorDistribution(range=8)

In [14]:
anim_frames = []
for step in range(num_steps):
    # update discriminator
    x_np = data.sample(batch_size)
    z_np = gen.sample(batch_size)    
    sess.run([loss_d, opt_disc], {x: np.reshape(x_np, (batch_size, 1)),z: np.reshape(z_np, (batch_size, 1))})

    # update generator
    z_np = gen.sample(batch_size)    
    sess.run([loss_g, opt_gen], {z: np.reshape(z_np, (batch_size, 1))})
    
    # write logs at every iteration
    summary = merged_summary_op.eval(
        feed_dict={x: np.reshape(x_np, (batch_size, 1)),z: np.reshape(z_np, (batch_size, 1))})
    summary_writer.add_summary(summary, step)
    
    # Handle animation    
    anim_frames.append(anim.samples(D_real, G, x, z, sess, data, gen.range, batch_size))

In [15]:
anim.save_animation(anim_frames, './plot.mp4', gen.range)